# Assignment 3 
## Data Preprocessing, PCA, and Modeling

DSCI 6601: Practical Machine Learning

Student: Sahil Khan

Date: 27-Oct-2024
_____________________________________________________________________________________________________________________________________________________________________________________________

### INTRODUCTION

In this assignment, we will explore various data pre-processing techniques followed by applying classification models to make predictions. This assignment will evaluate how data pre-processing and dimensionality reduction using PCA affects the performance of the models. The goal is to build a Decision Tree model for classification. We will evaluate the model’s performance and analyze how the features contribute to the predictions.

Dataset Link: https://www.kaggle.com/datasets/karrrimba/movie-metadatacsv

_____________________________________________________________________________________________________________________________________________________________________________________________

### IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay


import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)